# Flyer Basics

We want to understand how to build a *Flyer* in "Bluesky" to support various types of fly scans and remote data loggers.  The data about Flyers is spread about the standard documentation.  We need some clarity and a few examples that build complexity incrementally.

The basic notion of a Flyer is that it directs an *external controller* (we'll call it the *controller* here) to perform some data collection process.  Usually, a *controller* is used to collect data at rates beyond the capabilities of Bluesky plans and the RunEngine.  Examples could be waveforms from a storage oscilloscope or a continuous motion scan of a diffractometer.

This notebook will show the basic requirements for a Flyer and build a simple working example you can use a template for your own work.  (Our inspiration to create this [basic flyer tutorial](https://github.com/prjemian/ipython_mintvm/issues/3#issuecomment-385058577) was the goal of operating various fly scans at the APS, such as the USAXS fly scan, from Bluesky.  The examples we found before we started this project quickly became too instrument-specific to serve as tutorials.)
## Python imports and definitions 

Here are the full set of packages to imported.  The first block are Python standard packages, then come the ophyd, Bluesky, and databroker packages.  Just the parts we plan on using here.  Since this is also a tutorial, we will not rename imports or use other such shortcuts in the documentation (the online code has some shortcuts).

* Create a logger instance in case we want to investigate internal details as our code runs.
* Create an instance of the Bluesky RunEngine.
* Create an instance of the databroker using `databroker.temp()`, establishing a temporary, disposable yet fully-functional databroker for just this tutorial session.
* Subscribe that databroker instance to the RunEngine so it receives the document stream each time we run a plan.

Links

* USAXS fly scan - https://github.com/APS-USAXS/ipython-usaxs/issues/3
* `databroker.temp()` - https://blueskyproject.io/databroker/v1/api.html


In [1]:
import logging
import threading
import time

import ophyd
import bluesky
import bluesky.plans
import databroker

logger = logging.getLogger()
RE = bluesky.RunEngine({})
db = databroker.temp()
RE.subscribe(db.insert)

/home/travis/virtualenv/python3.7.1/lib/python3.7/site-packages/pims/image_reader.py:26: RuntimeWarning: PIMS image_reader.py could not find scikit-image. Falling back to matplotlib's imread(), which uses floats instead of integers. This may break your scripts. 
(To ignore this warning, include the line "warnings.simplefilter("ignore", RuntimeWarning)" in your script.)
  warnings.warn(RuntimeWarning(ski_preferred))


0

### Bare Minimum Requirements for a Flyer <a class="anchor" id="flyer-requirements" />

In Bluesky, a *Flyer* is an `ophyd.Device` that meets the *Fly-able interface*, which has three methods:

1. Kickoff - begin accumulating data
1. Complete - Bluesky tells the Flyer that Bluesky is ready to receive data
1. Collect - the device provides the data to Bluesky

The first two methods must return an instance of `ophyd.status.Status` (a.k.a. a *status* object).  For our code, we choose a specialized version of `Status()`, the `DeviceStatus()` object, based on the explanation provided in discussion.

The `collect()` method requires a companion `describe_collect()` that informs the RunEngine what kind of data to expect from `collect()`.  If no timestamp information is provided from the *controller*, then do as we show here and use the workstation's clock to provide a timestamp for the event.

This example (which does absolutely nothing) meets the bare minimum requirement.

Links

* *Flyer* - https://blueskyproject.io/bluesky/async.html?highlight=flyer#flying
* `ophyd.Device` - https://blueskyproject.io/ophyd/device-overview.html
* *Fly-able interface* - https://blueskyproject.io/ophyd/architecture.html#fly-able-interface
* returns - https://blueskyproject.io/bluesky/hardware.html#kickoff
* `ophyd.status.Status` - https://blueskyproject.io/ophyd/generated/ophyd.status.Status.html
* `DeviceStatus()` - https://blueskyproject.io/ophyd/generated/ophyd.status.DeviceStatus.html
* discussion - https://github.com/NSLS-II/tutorial-docker/pull/7#discussion_r185054364

In [2]:
class BareMinimumFlyer(ophyd.Device):

    def kickoff(self):
        kickoff_status = ophyd.DeviceStatus(self)
        kickoff_status._finished(success=True)
        return kickoff_status

    def complete(self):
        complete_status = ophyd.DeviceStatus(self)
        complete_status._finished(success=True)
        return complete_status

    def collect(self):
        yield {'data':{}, 'timestamps':{}, 'time':time.time()}
    
    def describe_collect(self):
        return {self.name: {}}


flyer = BareMinimumFlyer(name="flyer")
print(flyer.complete())
print(list(flyer.collect()))

# if this next step succeeds, it's proof that we did this right!
RE(bluesky.plans.fly([flyer]))

DeviceStatus(device=flyer, done=False, success=False)
[{'data': {}, 'timestamps': {}, 'time': 1593548154.0373235}]


/home/travis/virtualenv/python3.7.1/lib/python3.7/site-packages/event_model/__init__.py:172: UserWarning: The document type 'bulk_events' has been deprecated in favor of 'event_page', whose structure is a transpose of 'bulk_events'.
  "The document type 'bulk_events' has been deprecated in favor of "


('bc29021a-bf11-4885-b158-709c6a7f8086',)

Only three `documents` were emitted from the `RunEngine`: `start`, `descriptor`, and `stop` since we generated no data content.

In [3]:
list(db[-1].documents())

[('start',
  {'uid': 'bc29021a-bf11-4885-b158-709c6a7f8086',
   'time': 1593548154.0383663,
   'versions': {'ophyd': '1.5.1', 'bluesky': '1.6.3'},
   'scan_id': 1,
   'plan_type': 'generator',
   'plan_name': 'fly'}),
 ('descriptor',
  {'run_start': 'bc29021a-bf11-4885-b158-709c6a7f8086',
   'time': 1593548154.0409405,
   'data_keys': {},
   'uid': '10fe7b90-20ee-49c5-a0ca-d1cf917e60a5',
   'name': 'flyer',
   'configuration': {'flyer': {'data': {}, 'timestamps': {}, 'data_keys': {}}},
   'hints': {'flyer': {'fields': []}},
   'object_keys': {'flyer': []}}),
 ('stop',
  {'run_start': 'bc29021a-bf11-4885-b158-709c6a7f8086',
   'time': 1593548154.0424502,
   'uid': 'c0eb7dd8-acaf-4993-b40d-9777d8c8eb5a',
   'exit_status': 'success',
   'reason': '',
   'num_events': {'flyer': 1}})]

### Flyer : a starting template <a class="anchor" id="flyer-template" />

The `BareMinimumFlyer` is a good start to use a Flyer but we'll need to add a few more things to make a good template.  The first thing to do is to make the status objects known to any method of the class.  We'll prepend both of them with `self.` and describe in the code comments what is happening.  In the constructor (`__init__()`), we set both to `None`, the value we expect when not kicked off or *flying*, respectively.  Since we **need** a constructor, we must remember to call the constructor of the superclass as well or our `ophyd.Device` will not work correctly.

    def __init__(self, *args, **kwargs):
        super().__init__('', parent=None, **kwargs)
        self.kickoff_status = None
        self.complete_status = None

Our *controller* could take some time to signal that it is finished (seconds to minutes, at least).  We need a way to detect this completion.  We can do that either by polling the PV or by subscribing to a callback on the completion event.  We'll make that choice when we implement the actual activity.  Here, intend to wait in a polling loop.  Since the polling loop is an activity that does not return until the *controller* is done, we must do that waiting in a thread (consider an [alternative suggestion to use `epics.ca.CAThread` separate from that of the RunEngine.  (We do not want to block the RunEngine thread so it is free to respond to other activities, such as data from other streams or the user inerface.)  So, we run `my_activity()` in a separate thread that is called from `kickoff()`:

        thread = threading.Thread(target=self.my_activity, daemon=True)
        thread.start()

Link

* `epics.ca.CAThread` - https://github.com/NSLS-II/tutorial-docker/pull/7#discussion_r184905960

The basic outline of `my_activity()` is:

    def my_activity(self):
        # tell the *controller* to do its work
        # once started, we notify by updating the status object
        self.kickoff_status._finished(success=True)

        # wait for for it to tell us it is done 
        # after the wait, we declare victory
        self.complete_status._finished(success=True)

The waiting step will *block the thread* in which `my_activity()` is running but that's OK since this is separate from the RunEngine's thread.

Within `my_activity()`,  we've left two comments starting with `# TODO: ` marking where we need to add code that will complete the specifics of our final project.  Since this tutorial develops a starting template for that project (and others), we leave these comments as-is.

We've also added some diagnostic reporting (calls to `logger.info(...)`) to build out the next example:

In [4]:
class MyFlyer(ophyd.Device):
    """
    starting template for a Flyer that we understand
    """

    def __init__(self, *args, **kwargs):
        super().__init__('', parent=None, **kwargs)
        self.complete_status = None

    def my_activity(self):
        """
        start the "fly scan" here, could wait for completion
        
        It's OK to use blocking calls here 
        since this is called in a separate thread
        from the Bluesky RunEngine.
        """
        logger.info("activity()")
        if self.complete_status is None:
            logger.info("leaving activity() - not complete")
            return
        
        # TODO: do the activity here

        # once started, we notify by updating the status object
        self.kickoff_status._finished(success=True)

        # TODO: wait for completion
        
        # after the wait, we declare victory
        self.complete_status._finished(success=True)
        logger.info("activity() complete. status = " + str(self.complete_status))

    def kickoff(self):
        """
        Start this Flyer
        """
        logger.info("kickoff()")
        self.kickoff_status = ophyd.DeviceStatus(self)
        self.complete_status = ophyd.DeviceStatus(self)
        
        thread = threading.Thread(target=self.my_activity, daemon=True)
        thread.start()

        return self.kickoff_status

    def complete(self):
        """
        Wait for flying to be complete
        """
        logger.info("complete()")
        if self.complete_status is None:
            raise RuntimeError("No collection in progress")

        return self.complete_status

    def collect(self):
        """
        Retrieve the data
        """
        logger.info("collect()")
        self.complete_status = None
        yield {'data':{}, 'timestamps':{}, 'time':time.time()}
    
    def describe_collect(self):
        """
        Describe details for ``collect()`` method
        """
        logger.info("describe_collect()")
        return {self.name: {}}


In [5]:
ifly = MyFlyer(name="ifly")

### Diagnostics  <a class="anchor" id="Diagnostics" />

When building a `Flyer`, it is useful to have some diagnostics in place.  Already, we have been using some of these, including printing interim messages via calls to `logger.info(...)`.  Another useful diagnostic step is to call each of the methods individually to make sure they are acting as expected.

1. create an instance of the `Flyer`

    flyer = MyFlyer(name="flyer")

1. verify that `kickoff()` returns a status that is "Done"

    status = flyer.kickoff()
    status.done

1. verify that `complete()` returns a status that is "Done"

    status = flyer.complete()
    status.done

1. verify that `describe_collect()` returns a dictionary

    d = flyer.describe_collect()
    d

1. verify that `collect()` returns a generator

    g = flyer.collect()
    g

1. verify that generator is a list of data dictionaries

    list(g)


Apply some of those steps here (we'll skip testing the `ifly.complete()` method when not flying since it raises a `RuntimeError` exception if data collection is not in progress):

In [6]:
ifly.describe_collect()

{'ifly': {}}

In [7]:
list(ifly.collect())

[{'data': {}, 'timestamps': {}, 'time': 1593548154.1008072}]

Now, run this fly scan:

In [8]:
RE(bluesky.plans.fly([ifly]))

('8f95914d-1b0a-48db-8624-799b68f5c021',)

In [9]:
db[-1].stream_names

['ifly']

In [10]:
db[-1].table("ifly")

""
seq_num


Still only three `documents` were emitted from the `RunEngine`: `start`, `descriptor`, and `stop`.

In [11]:
list(db[-1].documents())

[('start',
  {'uid': '8f95914d-1b0a-48db-8624-799b68f5c021',
   'time': 1593548154.1087513,
   'versions': {'ophyd': '1.5.1', 'bluesky': '1.6.3'},
   'scan_id': 2,
   'plan_type': 'generator',
   'plan_name': 'fly'}),
 ('descriptor',
  {'run_start': '8f95914d-1b0a-48db-8624-799b68f5c021',
   'time': 1593548154.111055,
   'data_keys': {},
   'uid': '3dc51699-8580-45fb-8429-028769d0c0c7',
   'name': 'ifly',
   'configuration': {'ifly': {'data': {}, 'timestamps': {}, 'data_keys': {}}},
   'hints': {'ifly': {'fields': []}},
   'object_keys': {'ifly': []}}),
 ('stop',
  {'run_start': '8f95914d-1b0a-48db-8624-799b68f5c021',
   'time': 1593548154.1118486,
   'uid': '35a10b77-1421-4cf6-9903-740e7bf7c6c8',
   'exit_status': 'success',
   'reason': '',
   'num_events': {'ifly': 1}})]

## First working Flyer - trivial data <a class="anchor" id="trivial-data-flyer" />

To collect data, we need to modify both the `collect()` *and* the `describe_collect()` methods.  Bluesky needs to know what kind of data to expect from this Flyer, so that it can generate the correct `descriptor` document.

For the *most* trivial case, we'll return a single number (`1.2345`) as the result of the first working Flyer.

In the `describe_collect()` method, we create a dictionary that describes the data to be collected:

        d = dict(
            source = "fictional",
            dtype = "number",
            shape = []
        )
        return {
            'ifly': {
                "x": d
            }
        }

Then, in the `collect()` method, add the actual data collection code:

        t = time.time()
        d = dict(
            time=t,
            data=dict(x=1.2345),
            timestamps=dict(x=t)
        )
        yield d


In [12]:
class MyFlyer(ophyd.Device):
    """
    build a Flyer that we understand
    """

    def __init__(self, *args, **kwargs):
        super().__init__('', parent=None, **kwargs)
        self.complete_status = None

    def my_activity(self):
        """
        start the "fly scan" here, could wait for completion
        
        It's OK to use blocking calls here 
        since this is called in a separate thread
        from the Bluesky RunEngine.
        """
        logger.info("activity()")
        if self.complete_status is None:
            logger.info("leaving activity() - not complete")
            return
        
        # TODO: do the activity here

        # once started, we notify by updating the status object
        self.kickoff_status._finished(success=True)

        # TODO: wait for completion
        
        # after the wait, we declare victory
        self.complete_status._finished(success=True)
        logger.info("activity() complete. status = " + str(self.complete_status))

    def kickoff(self):
        """
        Start this Flyer
        """
        logger.info("kickoff()")
        self.kickoff_status = ophyd.DeviceStatus(self)
        self.complete_status = ophyd.DeviceStatus(self)
        
        thread = threading.Thread(target=self.my_activity, daemon=True)
        thread.start()

        return self.kickoff_status

    def complete(self):
        """
        Wait for flying to be complete
        """
        logger.info("complete()")
        if self.complete_status is None:
            raise RuntimeError("No collection in progress")

        return self.complete_status

    def describe_collect(self):
        """
        Describe details for ``collect()`` method
        """
        logger.info("describe_collect()")
        d = dict(
            source = "fictional",
            dtype = "number",
            shape = []
        )
        return {
            'ifly': {
                "x": d
            }
        }

    def collect(self):
        """
        Start this Flyer
        """
        logger.info("collect()")
        self.complete_status = None
        t = time.time()
        d = dict(
            time=t,
            data=dict(x=1.2345),
            timestamps=dict(x=t)
        )
        yield d

As before, create a new instance of the *revised* `MyFlyer` class.

In [13]:
ifly = MyFlyer(name="ifly")

In [14]:
print('output from describe_collect() : ', ifly.describe_collect())
print("list output from collect() : ", list(ifly.collect()))

output from describe_collect() :  {'ifly': {'x': {'source': 'fictional', 'dtype': 'number', 'shape': []}}}
list output from collect() :  [{'time': 1593548154.181204, 'data': {'x': 1.2345}, 'timestamps': {'x': 1593548154.181204}}]


Running this flyer with the RunEngine seems anticlimactic but the lack of exceptions tells us that it ran and we get a UUID at the end.

In [15]:
RE(bluesky.plans.fly([ifly]))

/home/travis/virtualenv/python3.7.1/lib/python3.7/site-packages/event_model/__init__.py:172: UserWarning: The document type 'bulk_events' has been deprecated in favor of 'event_page', whose structure is a transpose of 'bulk_events'.
  "The document type 'bulk_events' has been deprecated in favor of "


('286b2486-e000-4f4f-bb78-1691917866d8',)

We next query the last scan in the databroker and show a table of the stream from `collect()`:

In [16]:
h = db[-1]
h.table(h.stream_names[0])

,time,x
seq_num,,
1,2020-06-30 20:15:54.191664696,1.2345


Now, there is one more document type emitted: `event`.  Only one `event` since there is only one data point.

In [17]:
list(db[-1].documents())

[('start',
  {'uid': '286b2486-e000-4f4f-bb78-1691917866d8',
   'time': 1593548154.1889684,
   'versions': {'ophyd': '1.5.1', 'bluesky': '1.6.3'},
   'scan_id': 3,
   'plan_type': 'generator',
   'plan_name': 'fly'}),
 ('descriptor',
  {'run_start': '286b2486-e000-4f4f-bb78-1691917866d8',
   'time': 1593548154.1910713,
   'data_keys': {'x': {'source': 'fictional', 'dtype': 'number', 'shape': []}},
   'uid': '03fc94a6-d57c-4091-b1e8-c71b7157d4d9',
   'name': 'ifly',
   'configuration': {'ifly': {'data': {}, 'timestamps': {}, 'data_keys': {}}},
   'hints': {'ifly': {'fields': []}},
   'object_keys': {'ifly': ['x']}}),
 ('event',
  {'descriptor': '03fc94a6-d57c-4091-b1e8-c71b7157d4d9',
   'uid': 'a26a9315-9a0c-4256-bd18-152255ea937f',
   'time': 1593548154.1916647,
   'seq_num': 1,
   'data': {'x': 1.2345},
   'timestamps': {'x': 1593548154.1916647},
   'filled': {}}),
 ('stop',
  {'run_start': '286b2486-e000-4f4f-bb78-1691917866d8',
   'time': 1593548154.1920695,
   'uid': '97f9e7a5-5511

## Flyer that "collects" 1-D array data  <a class="anchor" id="simple-1d-array-flyer" />

We continue to expand the capabilities of our working example of a Flyer.  Now, we wish to "collect" a 1-D array of data.  We'll manufacture that data ourselves to keep the code simple.

Here, we generate an array of 5 numbers, each representing the elapsed time (in seconds) since the scan started (in `kickoff()`).  We record the start time with `self.t0 = time.time()`.  Also, `time.time()` is used to generate the timestamps in the data events.  In a real case, we *might* get timestamps from the *controller* or we'd have to make it up ourselves, just like this example.

In [18]:
class MyFlyer(ophyd.Device):
    """
    a Flyer that we understand that reports 1-D array of data
    """

    def __init__(self, *args, **kwargs):
        super().__init__('', parent=None, **kwargs)
        self.complete_status = None
        self.t0 = 0

    def my_activity(self):
        """
        start the "fly scan" here, could wait for completion
        
        It's OK to use blocking calls here 
        since this is called in a separate thread
        from the Bluesky RunEngine.
        """
        logger.info("activity()")
        if self.complete_status is None:
            logger.info("leaving activity() - not complete")
            return
        
        # TODO: do the activity here

        # once started, we notify by updating the status object
        self.kickoff_status._finished(success=True)

        # TODO: wait for completion
        
        # after the wait, we declare victory
        self.complete_status._finished(success=True)
        logger.info("activity() complete. status = " + str(self.complete_status))

    def kickoff(self):
        """
        Start this Flyer
        """
        logger.info("kickoff()")
        self.kickoff_status = ophyd.DeviceStatus(self)
        self.complete_status = ophyd.DeviceStatus(self)
        self.t0 = time.time()
        
        thread = threading.Thread(target=self.my_activity, daemon=True)
        thread.start()

        return self.kickoff_status

    def complete(self):
        """
        Wait for flying to be complete
        """
        logger.info("complete()")
        if self.complete_status is None:
            raise RuntimeError("No collection in progress")

        return self.complete_status

    def describe_collect(self):
        """
        Describe details for ``collect()`` method
        """
        logger.info("describe_collect()")
        d = dict(
            source = "elapsed time, s",
            dtype = "number",
            shape = (1,)
        )
        return {
            self.name: {
                "x": d
            }
        }

    def collect(self):
        """
        Start this Flyer
        """
        logger.info("collect()")
        self.complete_status = None
        for _ in range(5):
            t = time.time()
            x = t - self.t0 # data is elapsed time since kickoff()
            d = dict(
                time=t,
                data=dict(x=x),
                timestamps=dict(x=t)
            )
            yield d


In [19]:
ifly = MyFlyer(name="ifly")

In [20]:
print(ifly.describe_collect())

{'ifly': {'x': {'source': 'elapsed time, s', 'dtype': 'number', 'shape': (1,)}}}


In [21]:
list(ifly.collect())

[{'time': 1593548154.2655253,
  'data': {'x': 1593548154.2655253},
  'timestamps': {'x': 1593548154.2655253}},
 {'time': 1593548154.265528,
  'data': {'x': 1593548154.265528},
  'timestamps': {'x': 1593548154.265528}},
 {'time': 1593548154.2655294,
  'data': {'x': 1593548154.2655294},
  'timestamps': {'x': 1593548154.2655294}},
 {'time': 1593548154.2655306,
  'data': {'x': 1593548154.2655306},
  'timestamps': {'x': 1593548154.2655306}},
 {'time': 1593548154.2655323,
  'data': {'x': 1593548154.2655323},
  'timestamps': {'x': 1593548154.2655323}}]

Again, not much information from running this flyer, except that it succeeds and a uuid is returned.

In [22]:
RE(bluesky.plans.fly([ifly]))

/home/travis/virtualenv/python3.7.1/lib/python3.7/site-packages/event_model/__init__.py:172: UserWarning: The document type 'bulk_events' has been deprecated in favor of 'event_page', whose structure is a transpose of 'bulk_events'.
  "The document type 'bulk_events' has been deprecated in favor of "


('aed457a1-aa3d-479e-badc-3045d208ede1',)

More information is obtained by asking the databroker about the most recent scan.  Here's the data table from our stream of data events.  (The stream is named "ifly" as set from `self.name` in the `describe_collect()` method.)  We'll reference that last scan as `h` to reduce the amount of typing.

In [23]:
h = db[-1]
h.stream_names[0]

'ifly'

In [24]:
h.table(h.stream_names[0])

,time,x
seq_num,,
1,2020-06-30 20:15:54.277173281,0.001506
2,2020-06-30 20:15:54.277193546,0.001526
3,2020-06-30 20:15:54.277202368,0.001535
4,2020-06-30 20:15:54.277210712,0.001543
5,2020-06-30 20:15:54.277217865,0.001550


We generated 5 data points, there are 5 `event` documents to match.

In [25]:
list(db[-1].documents())

[('start',
  {'uid': 'aed457a1-aa3d-479e-badc-3045d208ede1',
   'time': 1593548154.2743971,
   'versions': {'ophyd': '1.5.1', 'bluesky': '1.6.3'},
   'scan_id': 4,
   'plan_type': 'generator',
   'plan_name': 'fly'}),
 ('descriptor',
  {'run_start': 'aed457a1-aa3d-479e-badc-3045d208ede1',
   'time': 1593548154.276599,
   'data_keys': {'x': {'source': 'elapsed time, s',
     'dtype': 'number',
     'shape': [1]}},
   'uid': '88d61c36-ef23-47ea-acf3-cd90cf45922f',
   'name': 'ifly',
   'configuration': {'ifly': {'data': {}, 'timestamps': {}, 'data_keys': {}}},
   'hints': {'ifly': {'fields': []}},
   'object_keys': {'ifly': ['x']}}),
 ('event',
  {'descriptor': '88d61c36-ef23-47ea-acf3-cd90cf45922f',
   'uid': 'a7a46675-44e0-4f96-9712-28bf42479601',
   'time': 1593548154.2771733,
   'seq_num': 1,
   'data': {'x': 0.0015056133270263672},
   'timestamps': {'x': 1593548154.2771733},
   'filled': {}}),
 ('event',
  {'descriptor': '88d61c36-ef23-47ea-acf3-cd90cf45922f',
   'uid': 'f6e2eba9-93

## Conclusion

Now you have seen the basic steps in creating a `Flyer`.  You have also been shown a few diagnostic tools to help you investigate your code development.